<a href="https://colab.research.google.com/github/ueofficer/AGU23_Workshop/blob/main/AGU23_WorkingFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **This notebook is developed for AGU 2023**

## Example 1 - Conductivity Temperature Depth

Data Product - Time Series Scalar Data (define *CTD, data columns,*)

Location - Folger Passage > Folger Deep

Period - 06-Jul-2013 to 07-Jul-2023

## Request Data Product

Jan 15, 9:05

In [1]:
import requests
import json
import pandas as pd

In [2]:
url = 'https://data.oceannetworks.ca/api/dataProductDelivery'
parameters = {'method':'request',
            'token':'7cc08ace-86d7-4136-9dcd-dd73017d35ae',
            'locationCode':'FGPD',
            'deviceCategoryCode':'CTD',
            'dataProductCode':'TSSD',
            'dateFrom':'2022-01-15T00:00:00.000Z',
            'dateTo':'2022-01-18T00:00:00.000Z',
            'extension':'csv',
            'dpo_qualityControl':'0',
            'dpo_dataGaps':'1',
            'dpo_resample':'none'} # replace YOUR_TOKEN_HERE with your personal token obtained from the 'Web Services API' tab at https://data.oceannetworks.ca/Profile when logged in.

response = requests.get(url,params=parameters)



In [3]:
if (response.ok):
    requestInfo = json.loads(str(response.content,'utf-8')) # convert the json response to an object

    print('Request Id: {}'.format(requestInfo['dpRequestId']))      # Print the Request Id

    if ('numFiles' in requestInfo.keys()):
        print('File Count: {}'.format(requestInfo['numFiles']))     # Print the Estimated File Size

    if ('fileSize' in requestInfo.keys()):
        print('File Size: {}'.format(requestInfo['fileSize']))      # Print the Estimated File Size

    if 'downloadTimes' in requestInfo.keys():
        print('Estimated download time:')
        for e in sorted(requestInfo['downloadTimes'].items(),key=lambda t: t[1]):
            print('  {} - {} sec'.format(e[0],'{:0.2f}'.format(e[1])))


    if 'estimatedFileSize' in requestInfo.keys():
        print('Estimated File Size: {}'.format(requestInfo['estimatedFileSize']))

    if 'estimatedProcessingTime' in requestInfo.keys():
        print('Estimated Processing Time: {}'.format(requestInfo['estimatedProcessingTime']))

else:
    if(response.status_code == 400):
        error = json.loads(str(response.content,'utf-8'))
        print(error) # json response contains a list of errors, with an errorMessage and parameter
    else:
        print ('Error {} - {}'.format(response.status_code,response.reason))

Request Id: 15089503
Estimated File Size: No estimated file size available.
Estimated Processing Time: No estimated processing time available.


In [4]:
import requests
import json

url = 'https://data.oceannetworks.ca/api/dataProductDelivery'
parameters = {'method':'run',
            'token':'7cc08ace-86d7-4136-9dcd-dd73017d35ae',              # replace YOUR_TOKEN_HERE with your personal token obtained from the 'Web Services API' tab at https://data.oceannetworks.ca/Profile when logged in.
           'dpRequestId':15089503}     # replace YOUR_REQUEST_ID_HERE with a requestId number returned from the request method
response = requests.get(url,params=parameters)

if (response.ok):
    r = json.loads(str(response.content,'utf-8')) # convert the json response to an object

    for runId in [run['dpRunId'] for run in r]:
        print('Run Id: {}'.format(runId))       # Print each of the Run Ids

else:
    if(response.status_code == 400):
        error = json.loads(str(response.content,'utf-8'))
        print(error) # json response contains a list of errors, with an errorMessage and parameter
    else:
        print ('Error {} - {}'.format(response.status_code,response.reason))

Run Id: 33587069


In [7]:
import requests
import json
import os
from contextlib import closing
import errno

url = 'https://data.oceannetworks.ca/api/dataProductDelivery'
parameters = {'method':'download',
            'token':'7cc08ace-86d7-4136-9dcd-dd73017d35ae',   # replace YOUR_TOKEN_HERE with your personal token obtained from the 'Web Services API' tab at https://data.oceannetworks.ca/Profile when logged in..
            'dpRunId':33587069,       # replace YOUR_RUN_ID with the dpRunId returned from the 'run' method.
            'index':1}                   # for run requests that contain more than one file, change the index number to the index of the file you would like to download.
                                           # If the index number does not exist an HTTP 410 and a message will be returned.


outPath='C:/Users/tricyaquino/Desktop/TRICY/UE-Officer/AD-HOC/AGU Workshop'                        # replace with the file location you would like the file to be downloaded to.

with closing(requests.get(url,params=parameters,stream=True)) as streamResponse:
    if streamResponse.status_code == 200: #OK
        if 'Content-Disposition' in streamResponse.headers.keys():
            content = streamResponse.headers['Content-Disposition']
            filename = content.split('filename=')[1]
        else:
            print('Error: Invalid Header')
            streamResponse.close()
            sys.exit(-1)

        filePath = '{}/{}'.format(outPath,filename)
        try:
            if (not os.path.isfile(filePath)):
                #Create the directory structure if it doesn't already exist
                try:
                    os.makedirs(outPath)
                except OSError as exc:
                    if exc.errno == errno.EEXIST and os.path.isdir(outPath):
                        pass
                    else:
                        raise
                print ("Downloading '{}'".format(filename))

                with open(filePath,'wb') as handle:
                    try:
                        for block in streamResponse.iter_content(1024):
                            handle.write(block)
                    except KeyboardInterrupt:
                        print('Process interupted: Deleting {}'.format(filePath))
                        handle.close()
                        streamResponse.close()
                        os.remove(filePath)
                        sys.exit(-1)
            else:
                print ("  Skipping '{}': File Already Exists".format(filename))
        except:
            msg = 'Error streaming response.'
            print(msg)
    else:
        if(streamResponse.status_code in [202,204,400,404,410]):
            payload = json.loads(str(streamResponse.content,'utf-8'))
            if len(payload) >= 1:
                msg = payload['message']
                print('HTTP {} - {}: {}'.format(streamResponse.status_code,streamResponse.reason,msg))
        else:
            print ('Error {} - {}'.format(streamResponse.status_code,streamResponse.reason))

streamResponse.close()